## RankForecasting-stint-paper

baseed on: 18/stint_test

stint rank prediction for paper



In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator as stint
import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlapstatus = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, model,trainid='2018')

    ret2 = {}
    
    #the simplest case, trim=0
    stint._trim = 0
    #stint._debug_carlist=[6]
    stint._debug_carlist=[]
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_pred(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

def test_pitmodelx(modelfile, testevent, runts, loopcnt=10, model='oracle'):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    if runts == 'RANK':
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                      pitmodel=modelfile, model=model)        
    else:
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                        'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model)

    #print(np.mean(acc, axis=0))
    return acc, ret

def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', inlapmode=0,featuremode = stint.FEATURE_PITAGE):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
                lapmode[inlapmode],
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    try:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)
    except:
        acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [3]:
acc, ret, pret = {}, {}, {}

In [4]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
                #if acc[mid] != []:
                #    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                #else:
                #    pret[mid] = []


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.79}, mae={3.11}, rmse={5.10},r2={0.37}, 
            naive: acc={0.17}, mae={5.72}, rmse={8.08},r2={0.29}
[[0.79130435 3.11304348 5.09503879 0.3655601 ]
 [0.17391304 5.72173913 8.08111056 0.28710352]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.80}, mae={3.09}, rmse={5.11},r2={0.35}, 
            naive: acc={0.17}, mae={5.72}, rmse={8.08},r2={0.29}
[[0.8        3.08985507 5.11264417 0.34830313]
 [0.17391304 5.72173913 8.08111056 0.28710352]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

In [5]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,featuremode = fmode)
                #if acc[mid] != []:
                #    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                #else:
                #    pret[mid] = []

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.56}, mae={5.22}, rmse={6.98},r2={-0.84}, 
            naive: acc={0.15}, mae={6.02}, rmse={8.31},r2={0.25}
model: acc={0.60}, mae={5.39}, rmse={7.22},r2={-1.19}, 
            naive: acc={0.15}, mae={6.01}, rmse={8.30},r2={0.26}
model: acc={0.57}, mae={5.34}, rmse={7.19},r2={-1.08}, 
            naive: acc={0.16}, mae={5.99}, rmse={8.29},r2={0.26}
[[ 0.57854534  5.3130495   7.12901064 -1.03537489]
 [ 0.15305408  6.01025008  8.29940648  0.25591416]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.52}, mae={5.95}, rmse={7.89},r2={-1.22}, 
            naive: acc={0.15}, mae={6.02}, rmse={8.31},r2={0.25}
model: acc={0.51}, mae={5.80}, rmse={7.66},r2={-1.07}, 
            naive: acc={0.16}, mae={5.99}, rmse={8.29},r2={0.26}
model: acc={0.50}, mae={5.95}, rmse={7.93},r2={-1.73}, 
            naive: acc={0.16}, mae={5.99}, rmse={8.29},r2={0.26}
[[ 0.50665729  5.90025641  7.82564247 -1.33943334]
 [ 0.15391495  6.00414009  8.29518625  0.25596508]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a sh

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a sh

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a sh

In [6]:
acc

{'oracle-TIMEDIFF-2018-noinlap-pitage': array([[[0.79130435, 3.11304348, 5.09503879, 0.3655601 ],
         [0.17391304, 5.72173913, 8.08111056, 0.28710352]]]),
 'oracle-TIMEDIFF-2018-inlap-pitage': array([[[0.8       , 3.08985507, 5.11264417, 0.34830313],
         [0.17391304, 5.72173913, 8.08111056, 0.28710352]]]),
 'oracle-TIMEDIFF-2018-outlap-pitage': [],
 'oracle-TIMEDIFF-2019-noinlap-pitage': array([[[0.71867008, 3.4168798 , 5.62375306, 0.09946494],
         [0.20971867, 5.40409207, 7.76953853, 0.31746095]]]),
 'oracle-TIMEDIFF-2019-inlap-pitage': array([[[0.75191816, 3.08184143, 5.40696718, 0.20236806],
         [0.20971867, 5.40409207, 7.76953853, 0.31746095]]]),
 'oracle-TIMEDIFF-2019-outlap-pitage': [],
 'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[0.7826087 , 4.15652174, 6.32547186, 0.36219144],
         [0.17391304, 5.72173913, 8.08111056, 0.28710352]]]),
 'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[0.8057971 , 3.14202899, 5.18246766, 0.42277607],
         [0.173913

In [7]:
sys.exit(0)

SystemExit: 0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
mid = 'oracle-2018'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1)
#_ = prisk(ret[mid][0][1], ret[mid][0][2])

#_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'oracle-2018-rank'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'RANK', loopcnt=1)

In [ ]:
mid = 'ranknet-2018-debug'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                   'Indy500-2018', 'TIMEDIFF', loopcnt=1)


In [ ]:
mid = 'ranknet-2018-rank-debug'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                   'Indy500-2018', 'RANK', loopcnt=1)

In [ ]:
mid = 'oracle-2019-timediff'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1)

In [ ]:
mid = 'oracle-2019-rank'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'RANK', loopcnt=1)

In [ ]:
mid = 'oracle-2018'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'RANK', loopcnt=1)

In [ ]:
mid = 'ranknet-2018'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK')


In [ ]:
mid = 'ranknet-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK')

In [ ]:
mid = 'ranknet-timediff-pit0-2018'
acc[mid], ret[mid] = test_pitmodel(0, 'Indy500-2018', 'TIMEDIFF',loopcnt=1)


In [ ]:
mid = 'ranknet-timediff-2018'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'TIMEDIFF',loopcnt=1)



In [ ]:
mid = 'ranknet-timediff-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'TIMEDIFF',loopcnt=1)



In [ ]:
#
# configurataiona
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df_oracle = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
df_oracle[df_oracle['carno']==12]

In [ ]:
stint.get_evalret(df_oracle)

### test

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')
df = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)
stint.get_evalret(df)


In [ ]:
allpits, pitmat, maxlap = stint.get_pitlaps(verbose=True)
#print(len(allpits), maxlap)
nextpit, maxnext = stint.get_nextpit(pitmat, 30)

print(nextpit, maxnext)

In [ ]:
len(pitmat)

In [ ]:
pitmat

In [ ]:
df

In [ ]:
df[df['carno']==12]

In [ ]:
correct = df[df['sign']==df['pred_sign']] 
acc = len(correct)/len(df)
acc

In [ ]:
stint.get_evalret(df)

In [ ]:
ret ={ }
ret['oracle'] = df
ret['curtrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTCURTRACK )

In [ ]:
stint.get_evalret(ret['curtrack'])

In [ ]:
ret['zerotrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTZERO )

In [ ]:
stint.get_evalret(ret['zerotrack'])

### test sim by pred

In [ ]:
def simulation2(datasetid, testevent, taskid, runts, expid, predictionlen, datamode, loopcnt, featuremode = stint.FEATURE_STATUS):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    stint.init()
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, 'oracle',trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        ret2[i] = stint.run_simulation_pred(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    return b

In [ ]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY, 3)

In [ ]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,3)


In [ ]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTCURTRACK,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTZERO,5)

In [ ]:
accret = simulation('indy2013-2018-pitage-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5, featuremode = stint.FEATURE_PITAGE)

In [ ]:
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
### pitmodel -- top8
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
### pitmodel -- top8
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

### sim pred

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()

#stint._dataset_id = 'indy2013-2018'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

#stint._dataset_id = 'indy2013-2018-nocarid-context40'
#stint._feature_mode = stint.FEATURE_STATUS
stint._dataset_id = 'indy2013-2018-pitage-nocarid-context40'
stint._feature_mode = stint.FEATURE_PITAGE

stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')
df2 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
stint.get_evalret(df2)
df3 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)
stint.get_evalret(df3)

In [ ]:
df2[df2['carno']==12]

In [ ]:
df2

In [ ]:
stint.get_evalret(df2)

In [ ]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
    
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
df[df['carno']==12]

In [ ]:
stint.get_evalret(df)

In [ ]:
acc = []
for i in ret.keys():
    df = ret[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

In [ ]:
a = [(1,2,3),(2,3,4)]
np.array(a)

In [ ]:
len(ret)

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
stint.get_evalret(df)

In [ ]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

### debug

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
stint.get_evalret(df)

### difference ? oracle .vs. pred

In [ ]:
#df, df2
df

In [ ]:
df2

In [ ]:
oracle = df[['carno','startlap']].values
pred =  df2[['carno','startlap']].values

In [ ]:
a = set(['%s-%s'%(x[0],x[1]) for x in oracle])
b = set(['%s-%s'%(x[0],x[1]) for x in  pred])
a -b

In [ ]:
df[df['carno']==3]

In [ ]:
df2[df2['carno']==3]

In [ ]:
_, data2018, _ = stint.load_data('Indy500-2018')
data2018[data2018['completed_laps']==200]

In [ ]:
topK = data2018[data2018['completed_laps']==200]['car_number'].values
topK

In [ ]:
topK_df = df[df['carno'].isin(topK[:5])]
stint.get_evalret(topK_df)

In [ ]:
len(topK_df)

In [ ]:
topK_df2 = df2[df2['carno'].isin(topK[:5])]
stint.get_evalret(topK_df2)

In [ ]:
len(topK_df2)

### check the dataset

In [ ]:
df = ret['oracle-2018']

In [ ]:
df = df[0]
df[df['carno']==6]

In [ ]:
len(df)

In [ ]:
dftemp = df[['carno','startlap', 'startrank','endrank']].sort_values(['carno','startlap'])
test = dftemp[['carno','startlap', 'startrank','endrank']].values.astype(int)
dfout = pd.DataFrame(test, columns =['carno','startlap', 'startrank','endrank'])
dfout.to_csv('test_db_ranknet2.csv')

In [ ]:
df = ret['ranknet-2018'][0]
df[df['carno']==6]

In [ ]:
len(df)

In [ ]:
ret['ranknet-timediff-2019']

In [ ]:
ret.keys()

In [ ]:
df=ret['oracle-2018'][0]
df.to_csv('result-stint')

In [10]:
df = ret['oracle-TIMEDIFF-2018-noinlap-pitage'][0]

In [11]:
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,27,11.0,20.0,9.0,1,20.0,9.0,1
1,13,28,20.0,12.0,-8.0,-1,13.0,-7.0,-1
2,14,28,5.0,10.0,5.0,1,10.0,5.0,1
3,59,28,32.0,32.0,0.0,0,32.0,0.0,0
4,3,29,6.0,10.0,4.0,1,8.0,2.0,1
...,...,...,...,...,...,...,...,...,...
340,25,194,0.0,16.0,16.0,1,16.0,16.0,1
341,59,194,20.0,21.0,1.0,1,21.0,1.0,1
342,60,194,1.0,17.0,16.0,1,17.0,16.0,1
343,19,195,9.0,15.0,6.0,1,16.0,7.0,1


In [12]:
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
15,12,30,1.0,3.0,2.0,1,2.0,1.0,1
30,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
71,12,48,5.0,5.0,0.0,0,5.0,0.0,0
92,12,49,5.0,0.0,-5.0,-1,0.0,-5.0,-1
169,12,92,0.0,0.0,0.0,0,0.0,0.0,0
177,12,93,0.0,0.0,0.0,0,0.0,0.0,0
235,12,127,0.0,1.0,1.0,1,0.0,0.0,0
240,12,128,1.0,0.0,-1.0,-1,0.0,-1.0,-1
309,12,169,0.0,7.0,7.0,1,3.0,3.0,1
